In [1]:
import networkx as nx
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import pandas as pd

from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *
from attention_graph_util import *
%matplotlib inline


import matplotlib as mpl
from scipy.stats import spearmanr
import math

[nltk_data] Downloading package punkt to /home/samira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load Task: VP

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    task_name = 'word_sv_agreement_vp'
    chkpt_dir='../tf_ckpts'
    task_params = get_task_params(batch_size=1)
    task = TASKS[task_name](task_params, data_dir='../data')
    cl_token = task.sentence_encoder().encode(constants.bos)
    tokenizer = task.sentence_encoder()._tokenizer


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Vocab len:  10032


In [3]:
#Load and evaluate a model

config = {'student_exp_name':'af_std5',
        'teacher_exp_name':'af_tchr5',
        'teacher_config':'small_lstm_v4',
        'student_model':'cl_bert',
        'teacher_model':'cl_lstm',
        'student_config':'small_gpt_v9',
        'distill_config':'dstl_6_crs_slw',
        'distill_mode':'online',
        'chkpt_dir':'../tf_ckpts',}

hparams=get_model_params(task, config['student_model'], config['student_config'])    
hparams.output_attentions = True
hparams.output_embeddings = True
hparams.output_hidden_states = True

with strategy.scope():
    model, ckpnt = get_student_model(config, task, hparams, cl_token)


model.evaluate(task.valid_dataset, steps=100)

model config: small_gpt_v9
model config: small_lstm_v4
student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/online_dstl_6_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_af_tchr5_student_cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_af_std5
Restored student from ../tf_ckpts/word_sv_agreement_vp/online_dstl_6_crs_slw_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_af_tchr5_student_cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_af_std5/ckpt-60


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


100/100 [==============================] - 5s 45ms/step - loss: 0.1045 - classification_loss: 0.0948 - sparse_categorical_accuracy: 0.9700


[0.10447468552738429, 0.09482438, 0.97]

In [ ]:
all_gradient_scores = []
all_inputgradient_scores = []
all_examples_x = []
all_examples_y = []
all_examples_attentions = []
all_examples_correct_probs = []
all_examples_correct_index_probs_diff = []
non_reshaped = []
n_batches = 10
prob_fn = task.get_probs_fn()
for x, y in tqdm(task.test_dataset):
    
    #Manually add cls token:
    batch_size = len(x)
    cl_token = tf.reshape(tf.convert_to_tensor(cl_token[0], dtype=tf.int64)[None], (-1,1))
    cl_tokens = tf.tile(cl_token, (batch_size, 1))
    x = tf.concat([cl_tokens, x], axis=-1)
    
    # Save examples
    all_examples_x.extend(x)
    all_examples_y.extend(y)
    
    # Get gradient scores
    input_embeddings, input_shape, padding_mask, past = model.get_input_embeddings(x, add_cls=False, training=False)
    with tf.GradientTape() as tape:
        tape.watch(input_embeddings)
        outputs = model.call_with_embeddings(input_embeddings, input_shape, padding_mask, past)
        logits = outputs[0]
        probs = tf.nn.softmax(logits, axis=-1)
        diff_probs = probs[:,0] - probs[:,1]
        
    grads = tape.gradient(diff_probs, input_embeddings)
    grad_scores = tf.abs(tf.reduce_sum(grads, axis=-1))
    input_grad_scores = tf.abs(tf.reduce_sum(tf.multiply(grads, input_embeddings), axis=-1))
    
    
    all_gradient_scores.extend(grad_scores)
    all_inputgradient_scores.extend(input_grad_scores)
    
    # Call the model to the get the logits and attentions
    outputs = model.detailed_call(x, add_cls=False, training=False)
    main_logits = outputs[0]
    
    # Get the probability of the correct class
    main_probs = prob_fn(main_logits, y, 1)
    main_diff_probs = main_probs[:,y[0]] #- main_probs[:,1]
    
    attentions_of_all_layers = outputs[6]

    
    # Reshape the attention matrix to: [batchsize, layers, heads, length, length]
    attentions_of_all_layers = [att.numpy() for att in attentions_of_all_layers]
    attentions_of_all_layers = np.transpose(np.asarray(attentions_of_all_layers), (1,0,2,3,4))
    
    # Save attentions and correct probs
    all_examples_attentions.extend(attentions_of_all_layers)

    
    # Repeating examples and replacing one token at a time with unk
    batch_size = tf.shape(x)[0]
    max_len = x.shape[1]
    
    # Repeat each example 'max_len' times
    extended_x = tf.reshape(tf.tile(x[:,None,...], (1,max_len, 1)),(-1,x.shape[-1]))
    extended_y = tf.reshape(tf.tile(y[:,None],(1,max_len)),(-1,))
    extended_diff_probs = tf.reshape(tf.tile(main_diff_probs[:,None],(1,max_len)),(-1,))
    
    # Create unk sequences and unk mask
    unktoken = 0 #task.databuilder.sentence_encoder().encode(constants.unk)
    unks = unktoken * tf.eye(max_len)
    unks = tf.cast(tf.tile(unks, (batch_size, 1)), dtype=tf.int64)
    unk_mask =  tf.cast((unktoken - unks)/unktoken, dtype=tf.int64)
  
    # Replace one token in each repeatition with unk
    extended_x = extended_x * unk_mask + unks
    
    # Get the new output
    extended_logits = model(extended_x, training=False, add_cls=False)
    extended_probs = prob_fn(extended_logits, extended_y, 1)

    
    extended_diff_probs = extended_probs[:,y[0]] # - extended_probs[:,1]
    # Save the difference in the probability predicted for the correct class
    diffs = abs(extended_diff_probs - main_diff_probs)
    diffs = tf.reshape(diffs,(batch_size,-1,1))
    all_examples_correct_index_probs_diff.extend(diffs)
    
    
    n_batches -= 1
    if n_batches <= 0:
        break
        

In [ ]:
print(all_examples_x[0].shape)
print(all_examples_correct_index_probs_diff[0].shape)
print(all_examples_attentions[0].shape)

In [ ]:
def my_spearmanr(x, y):
    """ `x`, `y` --> pd.Series"""
    x = pd.Series(x)
    y = pd.Series(y)
    assert x.shape == y.shape
    rx = x.rank(method='dense')
    ry = y.rank(method='dense')
    d = rx - ry
    dsq = np.sum(np.square(d))
    n = x.shape[0]
    coef = 1. - (6. * dsq) / (n * (n**2 - 1.))
    return [coef]

def get_raw_att_relevance(full_att_mat, input_tokens, layer=-1):
    cls_index = 0
    raw_rel = full_att_mat[layer].sum(axis=0)[cls_index]/full_att_mat[layer].sum(axis=0)[cls_index].sum()
    
    return raw_rel


def get_joint_relevance(full_att_mat, input_tokens, layer=-1):
    att_sum_heads =  full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
    relevance_attentions = joint_attentions[layer][0]
    return relevance_attentions


def get_flow_relevance(full_att_mat, input_tokens, layer):
    
    input_tokens = input_tokens
    res_att_mat = full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]

    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)
    
    A = res_adj_mat
    res_G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(res_G, {(i,j): A[i,j]}, 'capacity')


    output_nodes = ['L'+str(layer+1)+'_0']
    input_nodes = []
    for key in res_labels_to_index:
        if res_labels_to_index[key] < full_att_mat.shape[-1]:
            input_nodes.append(key)
    
    flow_values = compute_node_flow(res_G, res_labels_to_index, input_nodes, output_nodes, length=full_att_mat.shape[-1])
    
    length = full_att_mat.shape[-1]
    final_layer_attention = flow_values[(layer+1)*length:,layer*length:(layer+1)*length]
    cls_index = 0
    relevance_attention_flow = final_layer_attention[cls_index]

    return relevance_attention_flow

In [ ]:
for i in tqdm(np.arange(len(all_examples_x))):
        tokens = task.sentence_encoder().decode(all_examples_x[i]).split()
        length = len(tokens)
        all_gradient_scores[i] = all_gradient_scores[i][:length]
        all_inputgradient_scores[i] = all_inputgradient_scores[i][:length]

print(spearmanr(all_gradient_scores[0], all_inputgradient_scores[0]))

all_examples_raw_relevance = {}
for l in np.arange(0,6):
    all_examples_raw_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = task.sentence_encoder().decode(all_examples_x[i]).split()
        length = len(tokens)
        attention_relevance = get_raw_att_relevance(all_examples_attentions[i][...,:length, :length], tokens, layer=l)
        all_examples_raw_relevance[l].append(np.asarray(attention_relevance))

print(all_examples_raw_relevance[5][0].shape)


all_examples_blankout_relevance = {}
for l in np.arange(0,6):
    all_examples_blankout_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = task.sentence_encoder().decode(all_examples_x[i]).split()
        length = len(tokens)
        b_list = np.asarray(list(all_examples_correct_index_probs_diff[i].numpy().squeeze()))
        all_examples_blankout_relevance[l].append(b_list)
print(all_examples_blankout_relevance[5][0].shape)


all_examples_joint_relevance = {}
for l in np.arange(0,6):
    all_examples_joint_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = task.sentence_encoder().decode(all_examples_x[i]).split()
        length = len(tokens)
        attention_relevance = get_joint_relevance(all_examples_attentions[i][...,:length, :length], tokens, layer=l)
        all_examples_joint_relevance[l].append(np.asarray(attention_relevance))
print(all_examples_joint_relevance[5][0].shape)
    


In [ ]:
print(my_spearmanr(all_examples_blankout_relevance[5][8], all_examples_joint_relevance[5][8]))
print(spearmanr(all_examples_blankout_relevance[5][8], all_examples_joint_relevance[5][8]))

print(all_examples_blankout_relevance[5][8])
print(all_examples_joint_relevance[5][8])


print(len(all_examples_blankout_relevance[5][0]), len(all_examples_joint_relevance[5][0]))

In [ ]:
all_examples_flow_relevance = {}
for l in np.arange(0,6):
    all_examples_flow_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = task.sentence_encoder().decode(all_examples_x[i]).split()
        length = len(tokens)
        attention_relevance = get_flow_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_flow_relevance[l].append(np.asarray(attention_relevance))

print(all_examples_flow_relevance[5][0].shape)


In [ ]:

raw_sps_blank = []
raw_sps_grad = []
raw_sps_inputgrad = []

joint_sps_blank = []
joint_sps_grad = []
joint_sps_inputgrad = []

flow_sps_blank = []
flow_sps_grad = []
flow_sps_inputgrad = []


for l in np.arange(0,6):
    print("###############Layer ",l, "#############")
    print('raw blankout')
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_raw_relevance[l][i],all_examples_blankout_relevance[l][i])
        if not math.isnan(sp[0]):
            raw_sps_blank.append(sp[0])
        else:
            raw_sps_blank.append(0)
        
    print(np.mean(raw_sps_blank), np.std(raw_sps_blank))
    
    
    print('raw inputgrad')
    print(all_examples_raw_relevance[l][0].shape, all_inputgradient_scores[0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_raw_relevance[l][i],all_inputgradient_scores[i])
        if not math.isnan(sp[0]):
            raw_sps_inputgrad.append(sp[0])
        else:
            raw_sps_inputgrad.append(0)
        
    print(np.mean(raw_sps_inputgrad), np.std(raw_sps_inputgrad))
    
    print('raw grad')
    print(all_examples_raw_relevance[l][0].shape, all_gradient_scores[0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_raw_relevance[l][i],all_gradient_scores[i])
        if not math.isnan(sp[0]):
            raw_sps_grad.append(sp[0])
        else:
            raw_sps_grad.append(0)
        
    print(np.mean(raw_sps_grad), np.std(raw_sps_grad))
    
    print('************joint blankout')
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_joint_relevance[l][i],all_examples_blankout_relevance[l][i])
        if not math.isnan(sp[0]):
            joint_sps_blank.append(sp[0])
        else:
            joint_sps_blank.append(0)
        
    print(np.mean(joint_sps_blank), np.std(joint_sps_blank))
    
    print('joint grad')
    print(all_examples_joint_relevance[l][0].shape, all_gradient_scores[0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_joint_relevance[l][i],all_gradient_scores[i])
        if not math.isnan(sp[0]):
            joint_sps_grad.append(sp[0])
        else:
            joint_sps_grad.append(0)
        
    print(np.mean(joint_sps_grad), np.std(joint_sps_grad))
    
    print('joint inputgrad')
    print(all_examples_joint_relevance[l][0].shape, all_inputgradient_scores[0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_joint_relevance[l][i],all_inputgradient_scores[i])
        if not math.isnan(sp[0]):
            joint_sps_inputgrad.append(sp[0])
        else:
            joint_sps_inputgrad.append(0)
        
    print(np.mean(joint_sps_inputgrad), np.std(joint_sps_inputgrad))
    
    print('*************flow')
    print(all_examples_flow_relevance[l][0].shape,all_examples_blankout_relevance[l][0].shape)

    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_flow_relevance[l][i],all_examples_blankout_relevance[l][i])
        
        if not math.isnan(sp[0]):
            flow_sps_blank.append(sp[0])
        else:
            flow_sps_blank.append(0)
        
    print(np.mean(flow_sps_blank), np.std(flow_sps_blank))
  
    print('flow grad')
    print(all_examples_joint_relevance[l][0].shape, all_gradient_scores[0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_flow_relevance[l][i],all_gradient_scores[i])
        if not math.isnan(sp[0]):
            flow_sps_grad.append(sp[0])
        else:
            flow_sps_grad.append(0)
        
    print(np.mean(flow_sps_grad), np.std(flow_sps_grad))
    
    print('flow inputgrad')
    print(all_examples_joint_relevance[l][0].shape, all_inputgradient_scores[0].shape)
    for i in np.arange(len(all_examples_x)):
        sp = my_spearmanr(all_examples_flow_relevance[l][i],all_inputgradient_scores[i])
        if not math.isnan(sp[0]):
            flow_sps_inputgrad.append(sp[0])
        else:
            flow_sps_inputgrad.append(0)
        
    print(np.mean(flow_sps_inputgrad), np.std(flow_sps_inputgrad))
        

In [ ]:
print(len(all_examples_x))
for i in np.arange(len(all_examples_x)):
    print(all_examples_x[i].shape)
    print(all_examples_flow_relevance[i].shape)
    print(all_examples_raw_relevance[i].shape)
    print(all_examples_joint_relevance[i].shape)

In [ ]:
print(np.mean([my_spearmanr(all_examples_flow_relevance[5][i], all_examples_blankout_relevance[5][i]) for i in np.arange(len(all_examples_x))]))
print(np.mean([my_spearmanr(all_examples_joint_relevance[5][i], all_examples_blankout_relevance[5][i]) for i in np.arange(len(all_examples_x))]))
print(np.mean([my_spearmanr(all_examples_blankout_relevance[l][i], all_examples_blankout_relevance[l][i]) for i in np.arange(len(all_examples_x))]))

In [ ]:
print(np.mean([spearmanr(all_examples_blankout_relevance[l][i], all_examples_blankout_relevance[l][i]) for i in np.arange(len(all_examples_x))]))

for i in np.arange(len(all_examples_x)):
    print(all_examples_blankout_relevance[1][i])
    sp = spearmanr(all_examples_blankout_relevance[1][i], all_examples_blankout_relevance[1][i])
    print(sp)

In [ ]:
np.mean([spearmanr(all_examples_blankout_relevance[l][i], all_examples_blankout_relevance[l][i])[0] for i in np.arange(len(all_examples_x))])

In [37]:
model_1 = model

sentences = []
all_atts = []
all_main_probs = []
all_index_probs = []
all_gradient_scores = []
all_inputgradient_scores = []
prob_fn = task.get_probs_fn()
count = 0
for x, y in task.test_dataset:
    
     #Manually add cls token:
    batch_size = len(x)
    cl_token = tf.reshape(tf.convert_to_tensor(cl_token[0], dtype=tf.int64)[None], (-1,1))
    cl_tokens = tf.tile(cl_token, (batch_size, 1))
    x = tf.concat([cl_tokens, x], axis=-1)
    
    # Get gradient scores 
    input_embeddings, input_shape, padding_mask, past = model.get_input_embeddings(x, training=False, add_cls=False)
    with tf.GradientTape() as tape:
        tape.watch(input_embeddings)
        outputs = model_1.call_with_embeddings(input_embeddings, input_shape, padding_mask, past)
        logits = outputs[0]
        probs = tf.nn.softmax(logits, axis=-1)
        diff_probs = probs[:,0] - probs[:,1]
        
    grads = tape.gradient(diff_probs, input_embeddings)
    grad_scores = tf.abs(tf.reduce_sum(grads, axis=-1))
    input_grad_scores = tf.abs(tf.reduce_sum(tf.multiply(grads, input_embeddings), axis=-1))
    
    
    all_gradient_scores.extend(grad_scores)
    all_inputgradient_scores.extend(input_grad_scores)
    
    
    max_len = x.shape[1]
    all_outputs = model_1.detailed_call(x, training=False, add_cls=False)
    main_logits = all_outputs[0]
    attentions = all_outputs[6]
    _attentions = [att.numpy() for att in attentions]
    attentions = np.transpose(np.asarray(_attentions), (1,0,2,3,4))
    main_probs = prob_fn(main_logits, y, 1)
    batch_indexes = tf.range(len(y), dtype=tf.int64)
    indexes = tf.concat([batch_indexes[:,None], y[:,None]], axis=1)
    correct_main_probs = tf.gather_nd(main_probs, indexes).numpy()

    sentences.append(task.databuilder.sentence_encoder().decode(x[0]))
    all_atts.extend(attentions)
    all_main_probs.extend(correct_main_probs)
    all_index_probs.append([])
    
    # This loop can be optimized so that there is only one call...
    new_xz = []
    for i in np.arange(0,max_len):
        batch_size = tf.shape(x)[0]
        unktoken = task.databuilder.sentence_encoder().encode(constants.unk)
        unk = tf.reshape(tf.convert_to_tensor(unktoken, dtype=tf.int64)[None], (-1,1))
        unks = tf.tile(unk, (batch_size, 1))
        new_x = tf.concat([x[:,:i], unks, x[:,i+1:]], axis=-1)
        new_xz.extend(new_x)
    
    new_x = np.asarray(new_xz)
    logits = model_1(new_x, training=False, add_cls=False)
    probs = prob_fn(logits, y, 1)
    
    batch_indexes = tf.range(len(probs), dtype=tf.int64)
    yz = tf.tile(y, (len(probs),))

    indexes = tf.concat([batch_indexes[:,None], yz[:,None]], axis=1)
    
    correct_probs = tf.gather_nd(probs, indexes).numpy()
    all_index_probs[-1].extend(abs(correct_main_probs - correct_probs))
    count += 1
    if count > 100:
        break
    print (count, end="\r")

In [40]:
def spearmanr(x, y):
    """ `x`, `y` --> pd.Series"""
    x = pd.Series(x)
    y = pd.Series(y)
    assert x.shape == y.shape
    rx = x.rank(method='dense')
    ry = y.rank(method='dense')
    d = rx - ry
    dsq = np.sum(np.square(d))
    n = x.shape[0]
    coef = 1. - (6. * dsq) / (n * (n**2 - 1.))
    return coef

def get_raw_att_relevance(full_att_mat, input_tokens, layer=-1):
    cls_index = 0
    return full_att_mat[layer].sum(axis=0)[cls_index]
    

def compute_node_flow(G, labels_to_index, input_nodes, output_nodes,length):
    number_of_nodes = len(labels_to_index)
    flow_values=np.zeros((number_of_nodes,number_of_nodes))
    for key in output_nodes:
        if key not in input_nodes:
            current_layer = int(labels_to_index[key] / length)
            pre_layer = current_layer - 1
            u = labels_to_index[key]
            for inp_node_key in input_nodes:
                v = labels_to_index[inp_node_key]
                flow_value = nx.maximum_flow_value(G,u,v)
                flow_values[u][pre_layer*length+v ] = flow_value
            flow_values[u] /= flow_values[u].sum()
            
    return flow_values

def get_flow_relevance(full_att_mat, input_tokens, layer):
    
    res_att_mat = full_att_mat.sum(axis=1)/8
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]

    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)
    
    A = res_adj_mat
    res_G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(res_G, {(i,j): A[i,j]}, 'capacity')


    output_nodes = ['L'+str(layer+1)+'_0']
    input_nodes = []
    for key in res_labels_to_index:
        if res_labels_to_index[key] < full_att_mat.shape[-1]:
            input_nodes.append(key)
    
    flow_values = compute_node_flow(res_G, res_labels_to_index, input_nodes, output_nodes, length=full_att_mat.shape[-1])
    
    n_layers = full_att_mat.shape[0]
    length = full_att_mat.shape[-1]
    final_layer_attention_raw = flow_values[(layer+1)*length:,layer*length:(layer+1)*length]
    cls_index = 0
    relevance_attention_raw = final_layer_attention_raw[cls_index]

    return relevance_attention_raw
    
    
def get_joint_relevance(full_att_mat, input_tokens, layer):
    att_sum_heads =  full_att_mat.sum(axis=1)/8
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
    relevance_attentions = joint_attentions[layer][0]
    return relevance_attentions

for layer in np.arange(6):
    print("Layer: ", layer) 
    r_corels, r_pvals = [], []
    for i in np.arange(len(all_index_probs)):
        aa = np.asarray(all_index_probs[i])
        bb = get_raw_att_relevance(all_atts[i], ['<cls>']+sentences[i].split(), layer=layer)
        corel = spearmanr(aa,bb)
        r_corels.append(corel) 

    print("r:", np.mean(r_corels), np.std(r_corels))

    j_corels, j_pvals = [], []
    for i in np.arange(len(all_index_probs)):
        aa = all_index_probs[i]
        bb = get_joint_relevance(all_atts[i], ['<cls>']+sentences[i].split(),layer=layer)
        corel = spearmanr(aa,bb)
        j_corels.append(corel) 
    print("j:", np.mean(j_corels), np.std(j_corels))

    f_corels, f_pvals = [], []
    for i in np.arange(len(all_index_probs)):
        aa = all_index_probs[i]
        bb = get_flow_relevance(all_atts[i], ['<cls>']+sentences[i].split(),layer=layer)
        corel = spearmanr(aa,bb)
        f_corels.append(corel) 
    print("f:", np.mean(f_corels), np.std(f_corels))



Layer:  0
r: 0.457612570925546 0.2678428102423461
j: -0.1349056275417169 0.27168268088725456
f: -0.1349056275417169 0.27168268088725456
Layer:  1
r: 0.28168385315001815 0.3683819211171206
j: -0.07751297752096957 0.2883973956797265
f: 0.046081322087595196 0.3141916841909375
Layer:  2
r: -0.21467671772335364 0.4441958662880306
j: 0.1400533077143212 0.2637128279442795
f: 0.4720697083211908 0.2557136339407679
Layer:  3
r: -0.006999363758750784 0.4825294539189082
j: 0.2868584131286392 0.29004676602483825
f: 0.490094407447816 0.22219744351797496
Layer:  4
r: 0.21706006232836406 0.4043795055534174
j: 0.37544545323238876 0.27829844062549786
f: 0.4812995806167647 0.23327810924310977
Layer:  5
r: 0.2590064146637236 0.3913503072742342
j: 0.4106234744932413 0.2661281799443874
f: 0.49208497254385886 0.22065402275540336


In [42]:
for layer in np.arange(6):
    print("Layer: ", layer) 
    r_corels, r_pvals = [], []
    for i in np.arange(len(all_index_probs)):
        aa = all_gradient_scores[i]
        bb = get_raw_att_relevance(all_atts[i], ['<cls>']+sentences[i].split(), layer=layer)
        corel = spearmanr(aa,bb)
        r_corels.append(corel) 

    print("r:", np.mean(r_corels), np.std(r_corels))

    j_corels, j_pvals = [], []
    for i in np.arange(len(all_index_probs)):
        aa = all_gradient_scores[i]
        bb = get_joint_relevance(all_atts[i], ['<cls>']+sentences[i].split(),layer=layer)
        corel = spearmanr(aa,bb)
        j_corels.append(corel) 
    print("j:", np.mean(j_corels), np.std(j_corels))

    f_corels, f_pvals = [], []
    for i in np.arange(len(all_index_probs)):
        aa = all_gradient_scores[i]
        bb = get_flow_relevance(all_atts[i], ['<cls>']+sentences[i].split(),layer=layer)
        corel = spearmanr(aa,bb)
        f_corels.append(corel) 
    print("f:", np.mean(f_corels), np.std(f_corels))

Layer:  0
r: 0.5216894007179769 0.3041749647618828
j: 0.21413276141046322 0.2927459688754457
f: 0.21413276141046322 0.2927459688754457
Layer:  1
r: 0.15351287691264445 0.33859769357122405
j: 0.22958298335372324 0.31150738563851704
f: 0.31465241359682766 0.30240814789221254
Layer:  2
r: -0.022884233533028916 0.3950628027350199
j: 0.3715908698888184 0.2809389934389806
f: 0.5328858621912016 0.3020222138381862
Layer:  3
r: 0.012715486749277263 0.4790365003339604
j: 0.46007718115567814 0.2969363962997243
f: 0.6059368013333387 0.28208763085806887
Layer:  4
r: 0.24682037419674419 0.3791992915980523
j: 0.5182391931820621 0.30050598502505493
f: 0.6056054850657991 0.2728276667741501
Layer:  5
r: 0.4479414416224177 0.35342505569599236
j: 0.5313879704913244 0.29634751733627956
f: 0.6048833313416964 0.27519871426768183
